# Browyn Sherman DSTEP Final 

## Problem Statement

I am interested exploring education policy and school choice. Charter schools are one particular school choice option that have been a hot topic nationally. Charter schools are public schools that are independently operated. I am interested in exploring how charter schools compare to traditional public schools in terms of the populations they serve and the educational outcomes they achieve. I chose to focus on the state of Minnesota because they were the first state to authorize a public charter school in 1992. The Minnesota Department of Education collects data from schools and districts throughout the state and makes reports available to the public. Additional information about charter schools in the state of Minnesota is available here: https://education.mn.gov/mde/fam/cs/

In [1]:
# -- Import Useful Tools
import pandas as pd
import numpy as np
import geopandas as gpd
import time 

## Charter Schools

This dataset lists all charter schools in the state of MN: https://public.education.mn.gov/MdeOrgView/search/tagged/MDEORG_CHARTER_LEA,MARSSOperating 

In [2]:
# -- Load in and display list of charter schools 
fname = 'https://public.education.mn.gov/MdeOrgView/tag/extractContacts/MDEORG_CHARTER_LEA,MARSSOperating?description='
charters=pd.read_csv(fname)
charters

,District Number,District Type,School Number,Number,Organization,Title,Name,First Name,Last Name,Phone,...,Physical Zip,County,Grades,School Classification,Eco Dev Region,NCES ID,StateOrganizationId,ACT ID,Web URL,Data Extracted
0,4000,7,0,4000-07,City Academy,Director,Milo Cutter,Milo,Cutter,(651) 298-4624,...,55130-4058,Ramsey,9-12,0,11,2708955,74000000000,NaN,http://www.cityacademy.org/,11-08-2023
1,4001,7,0,4001-07,Bluffview Montessori,Director,Henry Schantzen,Henry,Schantzen,(507) 452-2807,...,55987-2459,Winona,KG-8,0,10,2705850,74001000000,NaN,http://www.bluffviewmontessori.org/,11-08-2023
2,4003,7,0,4003-07,"New Heights School, Inc.",Director,Thomas M. Kearney,Thomas,Kearney,(651) 439-1962,...,55082-4858,Washington,KG-12,0,11,2723380,74003000000,NaN,https://www.newheightsschool.com/,11-08-2023
3,4005,7,0,4005-07,Metro Deaf School,Director,Dr. Susan Outlaw,Susan,Outlaw,(651) 259-1050,...,55108-5032,Ramsey,EC-12,0,11,2700028,74005000000,NaN,http://www.mdsmn.org,11-08-2023
4,4007,7,0,4007-07,Minnesota New Country School,Director,Nancy Pfarr,Nancy,Pfarr,(507) 248-3353 x222,...,56044-0488,Sibley,KG-12,0,09,2700092,74007000000,NaN,http://www.newcountryschool.com,11-08-2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,4293,7,0,4293-07,Rollingstone Community School,Director,Elizabeth Nofsinger,Elizabeth,Nofsinger,(608) 344-0973,...,55969,Winona,KG-5,0,10,2700477,74293000000,NaN,https://rollingstonecommunityschool.com/,11-08-2023
188,4295,7,0,4295-07,Bultum Academy Charter School,Director,Arif Bakar,Arif,Bakar,(763) 568-4659,...,55421,Anoka,PK-8,0,11,2700481,74295000000,NaN,https://www.bultumacademy.org/,11-08-2023
189,4297,7,0,4297-07,Marine Village School,Director,Kimberly Kokx,Kimberly,Kokx,(651) 477-0380,...,55047,Washington,PK-8,0,11,2700483,74297000000,NaN,https://www.marinevillageschool.org/,11-08-2023
190,4298,7,0,4298-07,Endazhi-Nitaawiging,Director,Nathaniel Taylor,Nathaniel,Taylor,(218) 553-4265,...,56671,Beltrami,KG-8,0,02,2700484,74298000000,NaN,https://www.redlakecharterschool.com/,11-08-2023


## Geospatial Data

This source provides geospatial data for school districts in Minnesota: https://gisdata.mn.gov/dataset/bdry-school-district-boundaries

In [3]:
fname1='school_district_boundaries.shp'
districts=gpd.read_file(fname1)
districts

,SDORGID,FORMID,SDTYPE,SDNUMBER,PREFNAME,SHORTNAME,WEB_URL,SQMILES,ACRES,Shape_Leng,Shape_Area,geometry
0,1.000100e+10,0001-01,01,0001,Aitkin Public School District,Aitkin,https://home.isd1.org/,822.2109,526217.4564,273476.281014,2.129526e+09,"POLYGON ((438609.910 5121367.980, 438601.950 5..."
1,1.000200e+10,0002-01,01,0002,Hill City Public School District,Hill City,https://isd002.org/,259.1843,165878.7240,150623.758892,6.712873e+08,"POLYGON ((458092.000 5189241.000, 456458.000 5..."
2,1.000400e+10,0004-01,01,0004,McGregor Public School District,McGregor,https://www.mcgregor.k12.mn.us/,803.3305,514133.9628,327103.196022,2.080626e+09,"POLYGON ((489436.990 5111647.330, 487830.170 5..."
3,1.001100e+10,0011-01,01,0011,Anoka-Hennepin Public School District,Anoka-Hennepin,https://www.ahschools.us/,172.1338,110166.1810,148872.656027,4.458267e+08,"POLYGON ((477794.074 4991021.230, 477697.048 4..."
4,1.001200e+10,0012-01,01,0012,Centennial Public School District,Centennial,https://www.isd12.org/,27.8619,17831.6806,65474.900722,7.216224e+07,"MULTIPOLYGON (((493204.461 4996736.918, 493176..."
...,...,...,...,...,...,...,...,...,...,...,...,...
324,1.291000e+10,2910-01,01,2910,Ada-Borup-West Public Schools,Ada-Borup-West,http://ada.k12.mn.us/,532.6255,340881.9481,246104.342682,1.379500e+09,"POLYGON ((215650.909 5262658.690, 215672.910 5..."
325,2.032300e+10,0323-02,02,0323,Franconia Public School District,Franconia,NA,4.9802,3187.3229,18484.265585,1.289864e+07,"POLYGON ((517722.950 5015884.390, 516096.840 5..."
326,2.081500e+10,0815-02,02,0815,Prinsburg Public School District,Prinsburg,NA,27.9811,17907.9604,74820.233978,7.247094e+07,"POLYGON ((326152.631 4974862.470, 326553.883 4..."
327,3.000100e+10,0001-03,03,0001,Minneapolis Public School District,Minneapolis,http://www.mpls.k12.mn.us/,57.4692,36780.4309,60887.434019,1.488451e+08,"POLYGON ((476754.812 4970823.500, 476284.936 4..."


## Enrollment

These sources include student enrollment and demographic data from the 2013-2014 school year through the 2022-2023 school year https://public.education.mn.gov/MDEAnalytics/DataTopic.jsp?TOPICID=2. This data will be useful in looking at enrollment trends over time and whether charter schools serve the same populations as traditional public schools.

This data can be broken out by district type and charters are a district type: https://education.mn.gov/MDE/mde074910 This is useful because there have been new school openings and school closures during this timeframe.

In [4]:
# -- Load in Data by School 
f2014 = 'https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=MDE086207'
f2015 = 'https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=MDE086206'
f2016 = 'https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=MDE086205'
f2017 = 'https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=MDE086204'
f2018 = 'https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=MDE086203'
f2019 = 'https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=MDE086102'
f2020 = 'https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=MDE031850'
f2021 = 'https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=PROD034978'
f2022 = 'https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=PROD058443'
f2023 = 'https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=PROD070854'

In [5]:
sy14=pd.read_excel(f2014, sheet_name='School')
sy15=pd.read_excel(f2015, sheet_name='School')
sy16=pd.read_excel(f2016, sheet_name='School')
sy17=pd.read_excel(f2017, sheet_name='School')
sy18=pd.read_excel(f2018, sheet_name='School')
sy19=pd.read_excel(f2019, sheet_name='School')
sy20=pd.read_excel(f2020, sheet_name='School')
sy21=pd.read_excel(f2021, sheet_name='School')
sy22=pd.read_excel(f2022, sheet_name='School')
sy23=pd.read_excel(f2023, sheet_name='School')
sy14

,School - Enrollment Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59
0,Data Year,County Number,County Name,District Number,District Type,District Name,School Number,School Name,ECSU Number,Economic Development Region,...,Total Students Receiving Special Education Ser...,Total Students Receiving Special Education Ser...,Total English learner Identified Count,Total English learner Identified Percent,Total Students Experiencing Homelessness Count,Total Students Experiencing Homelessness Percent,Additional Suppression Free or Reduced-Priced ...,Additional Suppression Special Education Applied,Additional Suppression English learner Identif...,Additional Suppression Homeless Applied
1,13-14,01,Aitkin,0001,01,AITKIN PUBLIC SCHOOL DISTRICT,001,AITKIN SECONDARY SCHOOL,3,03,...,10,10.64%,0,0.00%,0,0.00%,N,N,N,N
2,13-14,01,Aitkin,0001,01,AITKIN PUBLIC SCHOOL DISTRICT,001,AITKIN SECONDARY SCHOOL,3,03,...,8,9.20%,0,0.00%,0,0.00%,N,N,N,N
3,13-14,01,Aitkin,0001,01,AITKIN PUBLIC SCHOOL DISTRICT,001,AITKIN SECONDARY SCHOOL,3,03,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,Y
4,13-14,01,Aitkin,0001,01,AITKIN PUBLIC SCHOOL DISTRICT,001,AITKIN SECONDARY SCHOOL,3,03,...,17,15.60%,0,0.00%,0,0.00%,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13219,13-14,48,Mille Lacs,6979,61,MID STATE EDUCATION DISTRICT,040,MID-STATE NORTH,83,05,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,Y
13220,13-14,48,Mille Lacs,6979,61,MID STATE EDUCATION DISTRICT,040,MID-STATE NORTH,83,05,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,Y
13221,13-14,48,Mille Lacs,6979,61,MID STATE EDUCATION DISTRICT,040,MID-STATE NORTH,83,05,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,Y
13222,13-14,48,Mille Lacs,6979,61,MID STATE EDUCATION DISTRICT,040,MID-STATE NORTH,83,05,...,NaN,>=90%,0,0.00%,0,0.00%,N,Y,N,N


In [6]:
# -- Load in Data for the State 
MN14=pd.read_excel(f2014, sheet_name='State')
MN15=pd.read_excel(f2015, sheet_name='State')
MN16=pd.read_excel(f2016, sheet_name='State')
MN17=pd.read_excel(f2017, sheet_name='State')
MN18=pd.read_excel(f2018, sheet_name='State')
MN19=pd.read_excel(f2019, sheet_name='State')
MN20=pd.read_excel(f2020, sheet_name='State')
MN21=pd.read_excel(f2021, sheet_name='State')
MN22=pd.read_excel(f2022, sheet_name='State')
MN23=pd.read_excel(f2023, sheet_name='State')
MN14

,State - Enrollment Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59
0,Data Year,County Number,County Name,District Number,District Type,District Name,School Number,School Name,ECSU Number,Economic Development Region,...,Total Students Receiving Special Education Ser...,Total Students Receiving Special Education Ser...,Total English learner Identified Count,Total English learner Identified Percent,Total Students Experiencing Homelessness Count,Total Students Experiencing Homelessness Percent,Additional Suppression Free or Reduced-Priced ...,Additional Suppression Special Education Applied,Additional Suppression English learner Identif...,Additional Suppression Homeless Applied
1,13-14,no data,no data,9999,99,Statewide,no data,no data,no data,no data,...,14551,99.97%,349,2.40%,104,0.71%,N,N,N,N
2,13-14,no data,no data,9999,99,Statewide,no data,no data,no data,no data,...,6772,10.32%,8253,12.58%,707,1.08%,N,N,N,N
3,13-14,no data,no data,9999,99,Statewide,no data,no data,no data,no data,...,7099,10.77%,8156,12.37%,719,1.09%,N,N,N,N
4,13-14,no data,no data,9999,99,Statewide,no data,no data,no data,no data,...,7228,11.20%,7924,12.27%,698,1.08%,N,N,N,N
5,13-14,no data,no data,9999,99,Statewide,no data,no data,no data,no data,...,8277,13.01%,7484,11.76%,638,1.00%,N,N,N,N
6,13-14,no data,no data,9999,99,Statewide,no data,no data,no data,no data,...,9358,14.63%,6402,10.01%,631,0.99%,N,N,N,N
7,13-14,no data,no data,9999,99,Statewide,no data,no data,no data,no data,...,9601,15.27%,5316,8.45%,549,0.87%,N,N,N,N
8,13-14,no data,no data,9999,99,Statewide,no data,no data,no data,no data,...,9184,14.85%,4844,7.83%,519,0.84%,N,N,N,N
9,13-14,no data,no data,9999,99,Statewide,no data,no data,no data,no data,...,9171,14.43%,4579,7.21%,512,0.81%,N,N,N,N


# Testing
This source has data on standardized testing results https://public.education.mn.gov/MDEAnalytics/DataTopic.jsp?TOPICID=1 and this source provides context: https://education.mn.gov/MDE/dse/test/mn/. This data may be useful in comparing academic outcomes between charters and traditional public schools.



In [7]:
# -- Load in Data by School 
math_file='https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=PROD082118'
reading_file='https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=PROD082119'
science_file='https://education.mn.gov/mdeprod/idcplg?IdcService=GET_FILE&RevisionSelectionMethod=latestReleased&Rendition=primary&dDocName=PROD082117'

In [8]:
math= pd.read_excel(math_file, sheet_name='School')
reading=pd.read_excel(reading_file, sheet_name='School')
science=pd.read_excel(science_file, sheet_name='School')
math

,Data Year,County Number,County Name,District Number,District Type,District Name,School Number,School Name,ECSU Number,Economic Development Region,...,Count Not Attempted,Count Not Complete,Count Not Enrolled,Count Refused,Count Refused - Parent,Count Refused - Student,Count Wrong Grade,Count Valid Scores MCA with Accommodations,Count Extenuating Circumstances Attempted,Count Extenuating Circumstances Not Attempted
0,22-23,1.0,Aitkin,1.0,1.0,Aitkin Public School District,1.0,Aitkin Secondary School,3,3,...,0.0,0.0,0.0,no data,0.0,0.0,0.0,0,0.0,0.0
1,22-23,1.0,Aitkin,1.0,1.0,Aitkin Public School District,1.0,Aitkin Secondary School,3,3,...,0.0,0.0,0.0,no data,0.0,0.0,0.0,1,0.0,0.0
2,22-23,1.0,Aitkin,1.0,1.0,Aitkin Public School District,1.0,Aitkin Secondary School,3,3,...,0.0,0.0,0.0,no data,6.0,0.0,0.0,0,0.0,0.0
3,22-23,1.0,Aitkin,1.0,1.0,Aitkin Public School District,1.0,Aitkin Secondary School,3,3,...,0.0,0.0,0.0,no data,6.0,0.0,0.0,1,0.0,0.0
4,22-23,1.0,Aitkin,1.0,1.0,Aitkin Public School District,2.0,Rippleside Elementary,3,3,...,0.0,0.0,0.0,no data,0.0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136736,22-23,49.0,Morrison,6979.0,61.0,Mid-State Education District,70.0,WIN-Middle School,5,5,...,0.0,0.0,0.0,no data,1.0,0.0,0.0,no data,0.0,0.0
136737,22-23,49.0,Morrison,6979.0,61.0,Mid-State Education District,70.0,WIN-Middle School,5,5,...,0.0,0.0,0.0,no data,0.0,0.0,0.0,no data,0.0,0.0
136738,22-23,49.0,Morrison,6979.0,61.0,Mid-State Education District,70.0,WIN-Middle School,5,5,...,0.0,0.0,0.0,no data,0.0,0.0,0.0,no data,0.0,0.0
136739,22-23,49.0,Morrison,6979.0,61.0,Mid-State Education District,70.0,WIN-Middle School,5,5,...,0.0,0.0,0.0,no data,1.0,0.0,0.0,no data,0.0,0.0


In [9]:
# -- Load in Data for the State 
MN_math= pd.read_excel(math_file, sheet_name='State')
MN_reading=pd.read_excel(reading_file, sheet_name='State')
MN_science=pd.read_excel(science_file, sheet_name='State')
MN_math

,Data Year,County Number,County Name,District Number,District Type,District Name,School Number,School Name,ECSU Number,Economic Development Region,...,Count Not Attempted,Count Not Complete,Count Not Enrolled,Count Refused,Count Refused - Parent,Count Refused - Student,Count Wrong Grade,Count Valid Scores MCA with Accommodations,Count Extenuating Circumstances Attempted,Count Extenuating Circumstances Not Attempted
0,22-23,no data,no data,9999.0,99.0,STATEWIDE,no data,no data,,,...,67.0,90.0,157.0,no data,792.0,33.0,2.0,1757.0,0.0,0.0
1,22-23,no data,no data,9999.0,99.0,STATEWIDE,no data,no data,,,...,65.0,92.0,138.0,no data,825.0,28.0,4.0,2130.0,0.0,0.0
2,22-23,no data,no data,9999.0,99.0,STATEWIDE,no data,no data,,,...,70.0,109.0,151.0,no data,997.0,37.0,2.0,2210.0,0.0,0.0
3,22-23,no data,no data,9999.0,99.0,STATEWIDE,no data,no data,,,...,109.0,164.0,165.0,no data,1239.0,67.0,1.0,2046.0,0.0,0.0
4,22-23,no data,no data,9999.0,99.0,STATEWIDE,no data,no data,,,...,169.0,222.0,234.0,no data,1857.0,151.0,2.0,1850.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,22-23,no data,no data,9999.0,99.0,STATEWIDE,no data,no data,,,...,169.0,222.0,233.0,no data,1846.0,151.0,2.0,1846.0,0.0,0.0
197,22-23,no data,no data,9999.0,99.0,STATEWIDE,no data,no data,,,...,183.0,251.0,284.0,no data,2875.0,240.0,5.0,1675.0,0.0,0.0
198,22-23,no data,no data,9999.0,99.0,STATEWIDE,no data,no data,,,...,1040.0,611.0,437.0,no data,8191.0,2658.0,16.0,833.0,0.0,0.0
199,22-23,no data,no data,9999.0,99.0,STATEWIDE,no data,no data,,,...,1701.0,1535.0,1551.0,no data,16752.0,3213.0,32.0,12481.0,0.0,0.0
